In [1]:
import torch
import numpy as np
import pandas as pd
import copy
import torch.nn.functional as F
import scipy.sparse as sp
from torch_geometric.nn import GCNConv,GATConv,SAGEConv
from torch_geometric.datasets import Planetoid
from torch.nn import Linear
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import datetime
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool

In [2]:
#device = "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
torch.cuda.set_device(1)
torch.cuda.current_device()

Using cpu device


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [3]:
device = cpu = torch.device('cpu')

In [ ]:
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
print (device)

In [4]:
from torch.nn import Linear
from torch_geometric.nn import GraphConv

class GNN(torch.nn.Module):
    def __init__(self, num_node_features,hidden_channels,num_classes):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels,num_classes)
        
    def forward(self, x, edge_index, batch, edge_weight=None):
        # 1. 获得节点嵌入
        x = self.conv1(x, edge_index,edge_weight)
        x = x.relu()
        x = self.conv2(x, edge_index,edge_weight)
        x = x.relu()
        x = self.conv3(x, edge_index,edge_weight)
        
        # 2. Readout layer
        x = global_mean_pool(x, batch)   # [batch_size, hidden_channels]
        
        # 3. 分类器
        x = F.dropout(x,p=0.5,training=self.training)
        x = self.lin(x)
        return x

In [ ]:
TORCH_USE_CUDA_DSA

In [5]:
## 读取reaction包含的基因
ReactionGeneFile = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.ProteinReactionNodes.txt"
ReactionGene = pd.read_csv(ReactionGeneFile,header=None,sep='\t')
ReactionGene.columns = ["Reaction","Gene"]
ReactionGene_dict = dict(zip(ReactionGene["Reaction"],ReactionGene["Gene"])) ## Reaction 对应的Gene
print (ReactionGene_dict['R-HSA-1112666'])
del ReactionGene

IGHV3-11,IGKV1D-33,SYK,SH3KBP1,IGHV1-69,IGLV1-47,IGKC,GRB2,IGHV2-70,IGLC7,IGKV3-11,IGHM,IGLV1-51,IGLV1-44,IGKV3-20,IGKV2D-28,IGKV2-30,NCK1,IGHV2-5,IGLC3,IGKV1-12,IGLV6-57,IGLV1-40,IGHD,IGKV2D-40,SOS1,IGLV3-25,IGHV3-23,IGHV1-2,IGLV2-14,IGKV2-28,IGKV1D-16,IGLV3-19,IGHV4-59,IGLV2-23,IGKV1D-12,IGHV3-33,IGLC1,BLNK,PLCG2,IGKV2D-30,IGLV3-27,IGKV3-15,IGLV2-11,IGHV3-48,IGHV3-9,BTK,IGLV2-8,IGKV1-17,IGKV3D-20,IGKV1-39,IGHV3-13,IGKV1-16,IGKV1D-39,IGHV4-34,IGKV2-29,IGLV7-43,IGLC6,IGKV4-1,IGLV3-1,IGKV5-2,CD79A,IGHV1-46,IGLV3-21,IGKV1-33,CD79B,IGKV1-5,IGHV4-39,VAV1,IGHV3-7,IGLC2,IGHV3-53,IGHV3-30


In [6]:
edges_file = "/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reactome_reaction.edges.txt"
edges = pd.read_csv(edges_file,header=None,sep="\t")
edges.columns = ["edge1","edge2","edge1_type","edge2_type"] 
uniq_output_nodes = sorted(list(set(edges["edge2"].tolist()))) ## 取得唯一入度的点
uniq_nodes = sorted(list(set(edges["edge1"].tolist()) | set(edges["edge2"].tolist())))
print (len(uniq_nodes))
edges = torch.load("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/Reactome_reaction_edges.pt")
print (edges)

11714
tensor([[11573, 11462,     3,  ...,  6095,  9849,  9046],
        [    0,     1,     2,  ..., 11713, 11713, 11713]])


In [7]:
## 读取 IBD不同药物反应的bulk数据
metainfo = pd.read_csv("/public/data/scDT/SingleCellDownload/IBD/Treatment_Bulk/Bulk_Anti-TNF/GSE16879/GSE16879.metainfo",header=0,sep=',')
#labels = ["Control","BeforeT_Yes","BeforeT_No"]
labels = ["BeforeT_Yes","BeforeT_No"]
metainfo = metainfo[metainfo["group"].isin(labels)].reset_index(drop=True)
bulk_exp = pd.read_csv("/public/data/scDT/SingleCellDownload/IBD/Treatment_Bulk/Bulk_Anti-TNF/GSE16879/GSE16879.exp.csv",header=0,sep=',',index_col=0)
bulk_exp = bulk_exp.T
print (bulk_exp.shape)
bulk_genelist = bulk_exp.columns
sample_full_labels = sorted(list(set(metainfo["group"].tolist())))
sample_full_labels_dict = dict(zip(sample_full_labels,range(len(sample_full_labels)))) ## label对应的index
sample_dict = dict(zip(metainfo["SampleName"],metainfo["group"]))
#list_labels = []
#for x in range(len(sample_full_labels)):
#    list_labels.append([x])
#list_labels_tensor = torch.tensor(list_labels,dtype=torch.int64)
print (sample_full_labels_dict[sample_dict["GSM364647"]])
#print (list_labels_tensor)

(133, 23520)
0


In [ ]:
sample_full_labels

In [ ]:
sample_full_labels_dict

In [8]:
print (bulk_exp.iloc[0:5,0:5])
select_sample = metainfo[metainfo["group"].isin(labels)]["SampleName"]
bulk_exp_select = bulk_exp[bulk_exp.index.isin(select_sample)]
sample_index_dict = dict(zip(bulk_exp_select.index,range(bulk_exp_select.shape[0]))) ## 将id 同index 存为字典
print (bulk_exp_select.shape)

GeneSymbol      A1BG  A1BG-AS1      A1CF       A2M   A2M-AS1
GSM364627   4.380101  4.419979  8.879925  6.624456  2.997803
GSM364628   3.772397  4.421832  8.280027  5.958338  3.119549
GSM364629   3.630654  4.297836  8.598125  6.577818  2.972677
GSM364630   3.779094  4.596442  9.035859  5.863000  2.786366
GSM364631   3.688025  4.582495  8.553964  5.824698  2.802469
(61, 23520)


In [9]:
## 对于每一个reaction , PCA 得到的主成分 PCA1值
import glob
import re
import os
import math
from sklearn import preprocessing

## 对于每一个reaction , PCA 得到的主成分 PCA1值
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(n_components=2)
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
sample_reaction_pca = pd.DataFrame()
n = 0
reaction_PCA_values = []
for node in iter(uniq_nodes): ## 计算所有reaction节点 对应的PCA1 values
    #print (node)
    #node = "R-HSA-1006169"
    if (node in ReactionGene_dict.keys()):
        genelist = ReactionGene_dict[node].split(",")
        common_gene = list(set(genelist) & set(bulk_genelist))
        ## 去除bulk中不存在的基因
        sub_exp = bulk_exp_select.loc[:,common_gene]
        #print (sub_exp)
        if (sub_exp.shape[1] >= 2):
            #sub_exp = sub_exp.values
            sub_exp = StandardScaler().fit_transform(sub_exp.values)
            principalComponents = pca.fit_transform(sub_exp)
            PCA_values =  pd.DataFrame(data = principalComponents,columns = ["PCA1","PCA2"])
            node_PCA_values = PCA_values["PCA1"]
        elif (sub_exp.shape[1] == 1): ## 如果reaction中只包含一个基因，无法做PCA，则对该基因表达量进行标准化
            sub_exp = StandardScaler().fit_transform(sub_exp.values)
            node_PCA_values = pd.DataFrame(data = sub_exp, columns=["PCA1"])
            #print (node_PCA_values)
        else:
            node_PCA_values = pd.DataFrame(np.zeros((bulk_exp_select.shape[0],1)),columns=["PCA1"])
    else:
        node_PCA_values = pd.DataFrame(np.zeros((bulk_exp_select.shape[0],1)),columns=["PCA1"])
    reaction_PCA_values.append(node_PCA_values)
    #sample_reaction_pca = pd.concat([sample_reaction_pca,node_PCA_values],axis=1,join='outer',ignore_index=True)
    #n += 1
    #if n >3:break
    #break
sample_reaction_pca = pd.concat(reaction_PCA_values,axis=1,join='outer',ignore_index=True)
print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
print (sample_reaction_pca.iloc[0:10,0:10])
print (sample_reaction_pca.shape)


2023-10-18  17:00:49
2023-10-18  17:01:31
          0         1         2         3         4         5         6  \
0 -1.360485 -0.966420  0.807790  0.807790  0.597877 -0.217869 -1.249992   
1  0.717703  0.350494 -2.191718 -2.191718 -0.254256 -0.526675 -0.430885   
2 -1.136742 -1.381867 -0.081990 -0.081990 -0.852702 -1.470781 -1.692378   
3 -0.508200 -1.551293 -0.863291 -0.863291 -0.859609 -1.892099 -2.228293   
4 -1.435456 -0.690978 -2.385350 -2.385350 -1.446189 -2.176223 -1.466531   
5 -0.191810 -0.623369 -0.596393 -0.596393 -0.310721 -0.221350 -0.381194   
6  1.216349 -0.413629  0.231225  0.231225 -0.885601 -0.382622  1.186200   
7  2.719679 -0.513786  0.034036  0.034036 -0.941039  0.612446  0.273110   
8 -0.458633 -0.150840 -0.571838 -0.571838 -0.545757  2.573742  1.445098   
9  1.482150 -0.035414  0.505400  0.505400 -0.846685  1.097248  0.755357   

          7         8         9  
0 -0.217869 -1.033377  1.046676  
1 -0.526675  0.313569 -0.795221  
2 -1.470781 -1.366306  1.12021

In [10]:
## 根据选取一半不同标签样本作为微调的训练集 
bulk_tuning_sets = []
bulk_test_sets = []
for group in labels:
    bulk_group_sample = metainfo[metainfo["group"] == group]["SampleName"]
    tuning_sample = bulk_group_sample.sample(math.ceil(bulk_group_sample.shape[0]/2)) ## 一半且向上取整
    print (tuning_sample)
    tuning_sample_index = tuning_sample.index
    print (group,len(tuning_sample))
    for m in tuning_sample:
        sampleName = m
        sample_index = sample_index_dict[sampleName]
        sample_value = sample_reaction_pca.loc[sample_index].values
        sample_value = sample_value.reshape(sample_value.shape[0],1)
        sample_node_feature = Data(x=torch.tensor(sample_value,dtype = torch.float32),y=torch.tensor(sample_full_labels_dict[sample_dict[sampleName]]),edge_index = edges)
        bulk_tuning_sets.append(sample_node_feature)
        
    test_sample = pd.concat([bulk_group_sample,tuning_sample,tuning_sample]).drop_duplicates(keep=False)
    for n in test_sample:
        sampleName = n
        sample_index = sample_index_dict[sampleName]
        sample_value = sample_reaction_pca.loc[sample_index].values
        sample_value = sample_value.reshape(sample_value.shape[0],1)
        sample_node_feature = Data(x=torch.tensor(sample_value,dtype = torch.float32),y=torch.tensor(sample_full_labels_dict[sample_dict[sampleName]]),edge_index = edges)
        bulk_test_sets.append(sample_node_feature)
        
print (len(bulk_tuning_sets))
print (len(bulk_test_sets))

47    GSM423061
6     GSM364639
31    GSM423023
2     GSM364635
29    GSM423019
44    GSM423055
26    GSM423013
33    GSM423027
27    GSM423015
35    GSM423031
32    GSM423025
49    GSM423065
28    GSM423017
0     GSM364633
Name: SampleName, dtype: object
BeforeT_Yes 14
52    GSM423071
37    GSM423035
14    GSM364647
56    GSM423079
39    GSM423039
23    GSM364656
42    GSM423045
21    GSM364654
53    GSM423073
9     GSM364642
19    GSM364652
8     GSM364641
15    GSM364648
36    GSM423033
58    GSM423083
51    GSM423069
57    GSM423081
Name: SampleName, dtype: object
BeforeT_No 17
31
30


In [ ]:
print (sample_full_labels_dict)
print (sample_index_dict["GSM364656"])
#print (sample_index_dict["GSM422967"])
print (sample_full_labels_dict[sample_dict["GSM364656"]])

In [11]:

def train_tuning():
    model_tuning.train() ## 
    for data in bulk_total_loader:
        #print (data.y)
        data = data.to(device)
        #x = data.x.to(device)
        #e = data.edge_index.to(device)
        #b = data.batch.to(device)
        #y = data.y.to(device)
        optimizer_tuning.zero_grad()
        out = model_tuning(data.x, data.edge_index,data.batch)
        loss = criterion_tuning(out, data.y)
        loss.backward()
        optimizer_tuning.step()
        
        
def test_tuning(loader):
    model_tuning.eval()
    correct = 0
    for data in loader:                           # 批遍历测试集数据集。
        data = data.to(device)
        out = model_tuning(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)   # 使用概率最高的类别
        #print (pred,data.y)
        correct += int((pred == data.y).sum())           # 检查真实标签
    return correct / len(loader.dataset)


In [15]:
#model_tuning = GNN(num_node_features=1,hidden_channels=64,num_classes=31)
#checkpoint = torch.load('/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reaction.PCA.pth.tar')
#model_tuning.load_state_dict(checkpoint["model"])
#model_tuning.eval()
model_tuning = torch.load("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/reaction.PCA_convEdgeWeight.1018.pt",map_location=device)
model_tuning.lin = Linear(64,2)
print (model_tuning)
for name,layer in model_tuning.named_parameters():
    if (name == 'lin.weight' or (name == 'lin.bias')):
        #print (name,layer)
        layer.requires_grad = True
    else:
        layer.requires_grad = False
model_tuning = model_tuning.to(device)
#model_tuning

GNN(
  (conv1): GraphConv(1, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [ ]:
for name, param in model_tuning.named_parameters(): print(name, param)

In [ ]:
model_tuning

In [13]:
#bulk_train_loader =  DataLoader(bulk_tuning_sets, batch_size=2,shuffle=True)
#bulk_test_loader = DataLoader(bulk_test_sets, batch_size=2,shuffle=True)
bulk_total_sets = bulk_tuning_sets + bulk_test_sets
bulk_total_loader = DataLoader(bulk_total_sets,batch_size=1,shuffle=False)
len(bulk_total_loader)

61

In [18]:
optimizer_tuning = torch.optim.AdamW(filter(lambda p: p.requires_grad, model_tuning.parameters()),lr=0.001)
criterion_tuning = torch.nn.CrossEntropyLoss()

print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))

for epoch in range(1, 201):
    train_tuning()
    print(datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S'))
    train_acc = test_tuning(bulk_total_loader)
    #test_acc = test_tuning(TCGA_test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
    if (train_acc > 0.85):
        break

2023-10-18  22:12:46
2023-10-18  22:12:47
Epoch: 001, Train Acc: 0.7541
2023-10-18  22:12:49
Epoch: 002, Train Acc: 0.7705
2023-10-18  22:12:52
Epoch: 003, Train Acc: 0.7377
2023-10-18  22:12:54
Epoch: 004, Train Acc: 0.7705
2023-10-18  22:12:56
Epoch: 005, Train Acc: 0.7705
2023-10-18  22:12:58
Epoch: 006, Train Acc: 0.7541
2023-10-18  22:13:00
Epoch: 007, Train Acc: 0.7869
2023-10-18  22:13:02
Epoch: 008, Train Acc: 0.7377
2023-10-18  22:13:05
Epoch: 009, Train Acc: 0.7541
2023-10-18  22:13:07
Epoch: 010, Train Acc: 0.7377
2023-10-18  22:13:09
Epoch: 011, Train Acc: 0.7377
2023-10-18  22:13:11
Epoch: 012, Train Acc: 0.7377
2023-10-18  22:13:13
Epoch: 013, Train Acc: 0.7541
2023-10-18  22:13:15
Epoch: 014, Train Acc: 0.7705
2023-10-18  22:13:17
Epoch: 015, Train Acc: 0.7541
2023-10-18  22:13:19
Epoch: 016, Train Acc: 0.7541
2023-10-18  22:13:21
Epoch: 017, Train Acc: 0.8197
2023-10-18  22:13:23
Epoch: 018, Train Acc: 0.7541
2023-10-18  22:13:25
Epoch: 019, Train Acc: 0.7541
2023-10-18

Epoch: 161, Train Acc: 0.8197
2023-10-18  22:17:46
Epoch: 162, Train Acc: 0.7705
2023-10-18  22:17:48
Epoch: 163, Train Acc: 0.7869
2023-10-18  22:17:50
Epoch: 164, Train Acc: 0.8033
2023-10-18  22:17:52
Epoch: 165, Train Acc: 0.8033
2023-10-18  22:17:53
Epoch: 166, Train Acc: 0.7705
2023-10-18  22:17:55
Epoch: 167, Train Acc: 0.7705
2023-10-18  22:17:57
Epoch: 168, Train Acc: 0.7377
2023-10-18  22:17:59
Epoch: 169, Train Acc: 0.7541
2023-10-18  22:18:00
Epoch: 170, Train Acc: 0.7541
2023-10-18  22:18:02
Epoch: 171, Train Acc: 0.7377
2023-10-18  22:18:04
Epoch: 172, Train Acc: 0.7869
2023-10-18  22:18:06
Epoch: 173, Train Acc: 0.7705
2023-10-18  22:18:08
Epoch: 174, Train Acc: 0.7213
2023-10-18  22:18:09
Epoch: 175, Train Acc: 0.7705
2023-10-18  22:18:11
Epoch: 176, Train Acc: 0.7705
2023-10-18  22:18:13
Epoch: 177, Train Acc: 0.7541
2023-10-18  22:18:15
Epoch: 178, Train Acc: 0.7377
2023-10-18  22:18:17
Epoch: 179, Train Acc: 0.7541
2023-10-18  22:18:18
Epoch: 180, Train Acc: 0.7541
2

In [ ]:
for name, param in model_tuning.named_parameters(): print(name, param)

In [ ]:
def test_tuning_subclass(loader):
    model_tuning.eval()
    correct = 0
    total = 0
    label_correct = {}
    total_label = {}
    for k in range(3):
        #print (type(k))
        total_label[k] = 0
        label_correct[k] = 0
    #print (label_correct[10])
    for data in loader:# 批遍历测试集数据集。
        data = data.to(device)
        out = model_tuning(data.x, data.edge_index, data.batch) # 一次前向传播
        pred = out.argmax(dim=1)   # 使用概率最高的类别
        total += int((pred == data.y).sum())
        index = 0
        for i in pred:
            num = i.item()
            #print (num)
            #print (data.y[0].item())
            total_label[data.y[index].item()] += 1
            if (num == data.y[index].item()):
                label_correct[num] += 1
            index += 1
    for j in total_label.keys():
        correct_num = label_correct[j]
        ratio = correct_num / total_label[j] if total_label[j] != 0 else 0
        print (j,total_label[j],ratio)
    return total / len(loader.dataset)

In [ ]:
test_acc = test_tuning_subclass(bulk_test_loader)
#print (test_acc.)
print(f'Test Acc: {test_acc:.4f}')

In [ ]:
from captum.attr import Saliency, IntegratedGradients

In [ ]:
def model_forward(e_mask, dt):
    #batch = torch.zeros(dt.x.shape[0], dtype=int).to(device)
    batch = torch.zeros(dt.x.shape[0], dtype=int).to(device)
    out = model_tuning(dt.x,
                dt.edge_index,
                batch,
                e_mask)
    return out

In [ ]:
def explain(method, dt, target=0):
    #input_mask = torch.ones(dt.edge_index.shape[1]).requires_grad_(True).to(device)
    input_mask = torch.ones(dt.edge_index.shape[1]).requires_grad_(True)
    if method == 'ig':
        ig = IntegratedGradients(model_forward)
        mask = ig.attribute(input_mask, target=target,
                            additional_forward_args=(dt,),
                            internal_batch_size=dt.edge_index.shape[1])
    elif method == 'saliency':
        saliency = Saliency(model_forward)
        mask = saliency.attribute(input_mask, target=target,
                                  additional_forward_args=(data,))
    else:
        raise Exception('Unknown explanation method')
 
    edge_mask = np.abs(mask.cuda().detach().numpy())
    if edge_mask.max() > 0:  # avoid division by zero
        edge_mask = edge_mask / edge_mask.max()
    return edge_mask

In [ ]:
data = bulk_total_sets[10]
print (data)
data = data.to(device)
target = 0
target = torch.tensor(target).to(device)
#import random
#data = random.choice([t for t in bulk_test_sets if not t.y.item()])

In [ ]:
target

In [ ]:
torch.save(model_tuning, '/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/GSE16879.IBD.Response.pt')

In [ ]:
torch.save(bulk_total_sets,"/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/GSE16879.IBD.Response.data.pt")

In [ ]:
edge_mask = explain("ig", data, target=0)


In [ ]:
for target_tissue in range(2):
    title = 'Integrated Gradients'
    method = 'ig'
    #data.to(device)
    #print(F"processing tissue {target_tissue} with {title}, a.k.a. {method}")
    edge_mask = explain(method, data, target=target_tissue)
    # edge_mask_dict = aggregate_edge_directions(edge_mask, data)
    #path = F"/home/jgburk/PycharmProjects/reticula/data/SRP035988/output/{method}_{target_tissue}.txt"
    #numpy.savetxt(path, edge_mask, delimiter=",")
    #print(F"{method} {target_tissue} edges saved as {path}")

In [ ]:
#input_mask = torch.ones(data.edge_index.shape[1]).requires_grad_(True).to(device)
input_mask = torch.ones(data.edge_index.shape[1]).requires_grad_(True)
input_mask

In [ ]:
ig = IntegratedGradients(model_forward)
ig.attribute

In [ ]:
mask = ig.attribute(input_mask, target=1,
                            additional_forward_args=(data,),
                            internal_batch_size=data.edge_index.shape[1])
mask

In [ ]:
import sklearn
t_v = np.loadtxt("/public/home/liujunwu/workdir/scripts/GNN_Reactome/reaction_file/test.label", dtype=str)
t_v
t_encoder = sklearn.preprocessing.LabelEncoder()
t_v = t_encoder.fit_transform(t_v)
torch.tensor(t_v[1])

In [ ]:
data1 = DataLoader(bulk_total_sets)

In [ ]:
print (data.edge_index.device)
print (data.x.device)
print (data.y.device)

In [ ]:
data1.dataset[0].edge_index